# Exercise: Generators and iterators

## About
This excersise trains you about Python generators and iterators.

## Tasks

1. Execute the following generator example and note the difference in memory usage between standard and lazy evaluations.

  [Learing objectives: generators, generator comprehension]

2. Create your own iterator object behaving like an earlier generator example.

  [Learing objectives: iterators]

## 1. Generator example

Assume we are writing a monte-carlo event generator that produces very large objects.

In [ ]:
import numpy as np
import sys

def make_event():
    huge_readout = np.random.uniform(size=10000000)
    return huge_readout

event = make_event()
print(f"Size in Mb: {sys.getsizeof(event)/1024**2 : .2f}")

Size in Mb:  76.29


Now assume that we want to generate 100 events and calculate some summary statistic for each event. If we would loop over our `make_event` function and collect the resulting events in a list, we would quickly run out of memory.

In [ ]:
def summary_stat(event):
    return np.sum(event)

n_events = 10
# this will blow up your memory with e.g. `n_events = 100`
events = [make_event() for i in range(n_events)]
print(f"Size in Mb: {len(events) * sys.getsizeof(event)/1024**2 : .2f}")

summary_stats = [summary_stat(event) for event in events]
print(f"Median of summary_stats: {np.median(summary_stats):.2f}")

Size in Mb:  762.94
Median of summary_stats: 4999987.67


We can solve this issue by e.g. using a generator.

In [ ]:
def make_events(n_events):
    for i in range(n_events):
        yield make_event()
      
n_events = 100
generator = make_events(n_events)
print(f"Size in **bytes**: {sys.getsizeof(generator): .2f}")

summary_stats = [summary_stat(event) for event in generator]
print(f"Median of summary_stats: {np.median(summary_stats):.2f}")

Size in **bytes**:  128.00
Median of summary_stats: 5000038.29


Achieve the same functionality as `make_events` function by using generator comprehension.

In [ ]:
n_events = 100

# Note that we use round brackets for generator comprehension
# in comparison to square brackets for list comprehension.
generator = (make_event() for i in range(n_events))
print(f"Size in **bytes**: {sys.getsizeof(generator): .2f}")

summary_stats = [summary_stat(event) for event in generator]
print(f"Median of summary_stats: {np.median(summary_stats):.2f}")

Size in **bytes**:  128.00
Median of summary_stats: 4999949.47


## 2. Iterator exercise

Create your own iterator object behaving like an earlier generator example.

In [ ]:
# Initial code example. 
# Replace pass statements with your code.

class MakeEvents(object):
    def __init__(self, n_events):
        self.n_events = n_events
        self.n_generated = 0

    def __iter__(self):
        return self

    def __next__(self):
        if self.n_generated == self.n_events:
            raise StopIteration()
        self.n_generated += 1
        return make_event()

Use the following cell to test your iterator.

In [ ]:
n_events = 100

iterator = MakeEvents(n_events)

summary_stats = [summary_stat(event) for event in iterator]
print(f"Median of summary_stats: {np.median(summary_stats):.2f}")

# Additional check if the iterator produces correct number of events.
if len(summary_stats) != n_events:
    raise ValueError("Lenght of generated events in `summary_stats` is not equal to `n_events`!")

Median of summary_stats: 4999996.25
